In [2]:
# Loading necessary libraries

import pandas as pd

# Loading the dataset
data = pd.read_csv("TRADES_CopyTr_90D_ROI.csv")
print(data.head())

              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


In [3]:
# Check for missing values
print(data.isnull().sum())

Port_IDs         0
Trade_History    1
dtype: int64


In [4]:
#Drop rows with missing Trade values
data = data.dropna(subset="Trade_History")

In [5]:
# Convert Trade_History from string to list of dictonaries

import ast
data["Trade_History"] = data["Trade_History"].apply(ast.literal_eval)

In [6]:
# Flatten Trade_History into a single DataFrame
trades = []
for index, row in data.iterrows():
    port_id = row["Port_IDs"]
    for trade in row["Trade_History"]:
        trade["Port_ID"] = port_id
        trades.append(trade)

trades_df = pd.DataFrame(trades)
print(trades_df.head())

            time    symbol side      price       fee feeAsset    quantity  \
0  1718899656000   SOLUSDT  BUY  132.53700 -0.994027     USDT  1988.05500   
1  1718899618000  DOGEUSDT  BUY    0.12182 -0.279796     USDT  1398.98088   
2  1718899618000  DOGEUSDT  BUY    0.12182 -0.039494     USDT   197.47022   
3  1718899616000  DOGEUSDT  BUY    0.12182 -0.008284     USDT    16.56752   
4  1718899616000  DOGEUSDT  BUY    0.12182 -0.046109     USDT    92.21774   

  quantityAsset  realizedProfit realizedProfitAsset baseAsset      qty  \
0          USDT             0.0                USDT       SOL     15.0   
1          USDT             0.0                USDT      DOGE  11484.0   
2          USDT             0.0                USDT      DOGE   1621.0   
3          USDT             0.0                USDT      DOGE    136.0   
4          USDT             0.0                USDT      DOGE    757.0   

  positionSide  activeBuy              Port_ID  
0         LONG       True  3925368433214965

In [11]:
# Group by Port_ID and calculated metrics
metrics = []

for port_id, group in trades_df.groupby("Port_ID"):
    total_invested = group["quantity"].sum()
    total_pnl = group["realizedProfit"].sum()
    roi = total_pnl/total_invested if total_invested !=0 else 0

    daily_returns = group.groupby(group["time"].astype(int) // (24*3600*1000))["realizedProfit"].sum()
    avg_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()
    sharpe_ratio = avg_daily_return / std_daily_return if std_daily_return != 0 else 0

    cumulative_profit = group["realizedProfit"].cumsum()
    peak = cumulative_profit.cummax()
    drawdown = peak - cumulative_profit
    mdd = drawdown.max()

    win_positions = (group["realizedProfit"] > 0).sum()
    total_positions = len(group)
    win_rate = win_positions / total_positions if total_positions != 0 else 0

    metrics.append({
        "Port_ID": port_id,
        "ROI" : roi,
        "PnL": total_pnl,
        "Sharpe_Ratio": sharpe_ratio,
        "MDD": mdd,
        "Win_Rate": win_rate,
        "Win_Positions": win_positions,
        "Total_Positions": total_positions
    })

metrics_df = pd.DataFrame(metrics)
print(metrics_df.head())

               Port_ID       ROI          PnL  Sharpe_Ratio          MDD  \
0  3672754654734989568  0.004764   566.597660      0.457266    80.777037   
1  3733192481840423936  0.002511  2923.977200      0.194478  2743.120000   
2  3768170840939476993  0.087791   243.668899      0.707107     0.000000   
3  3784403294629753856  0.003398  2521.814305      0.696929   271.291981   
4  3786761687746711808  0.003321   205.021400      0.260437   123.460267   

   Win_Rate  Win_Positions  Total_Positions  
0  0.443038            210              474  
1  0.802612            553              689  
2  0.428571              6               14  
3  0.302314           1829             6050  
4  0.451220             37               82  


In [12]:
# Define weights
weights = {
    "ROI": 0.3,
    "Sharpe_Ratio": 0.25,
    "MDD": -0.2,
    "Win_Rate": 0.15,
    "Win_Positions": 0.1,
    "Total_Positions": 0.1
}

# Normalize metrics
for col in ["ROI", "Sharpe_Ratio", "MDD", "Win_Rate", "Win_Positions", "Total_Positions"]:
    metrics_df[col] = (metrics_df[col] - metrics_df[col].min()) / (metrics_df[col].max() - metrics_df[col].min())

# Calculate composite score
metrics_df["Score"] = (
    metrics_df["ROI"] * weights["ROI"] +
    metrics_df["Sharpe_Ratio"] * weights["Sharpe_Ratio"] +
    metrics_df["MDD"] * weights["MDD"] +
    metrics_df["Win_Rate"] * weights["Win_Rate"] +
    metrics_df["Win_Positions"] * weights["Win_Positions"] +
    metrics_df["Total_Positions"] * weights["Total_Positions"]
)

# Rank accounts by score
metrics_df = metrics_df.sort_values(by="Score", ascending=False)
top_20_accounts = metrics_df.head(20)
print(top_20_accounts[["Port_ID", "Score"]])

                 Port_ID     Score
97   4021243448368889856  0.501333
8    3826087012661391104  0.497947
62   3986814617275053313  0.441930
100  4022641794255717633  0.415882
37   3944088772635000577  0.396387
16   3891020560590657281  0.393572
2    3768170840939476993  0.390439
33   3939925275039085824  0.371659
17   3907081197088384000  0.369893
15   3887577207880438784  0.367115
75   3999240873283311617  0.360909
86   4011626972687000576  0.342966
27   3931992636670880512  0.333431
48   3956076827719377409  0.327480
14   3886752488982104320  0.322284
76   3999845462424248576  0.322224
3    3784403294629753856  0.321304
36   3943533600390906881  0.303893
112  4029749871687083265  0.299113
92   4017110277719148289  0.293682


In [13]:
# Saveing the metrics to csv file

metrics_df.to_csv("calculated_metrics.csv", index=False)